In [ ]:
# Install the pytorch and ultralytics 
%pip install -U ultralytics
%pip install torch torchvision torchaudio
%pip install pandas

In [1]:
import ultralytics 
#import cv2
import pandas as pd
from distutils.dir_util import copy_tree
import os
# Check  GPU  and pytorch works
ultralytics.checks()

# It should list GPU device ex:  
# Ultralytics YOLOv8.1.0 🚀 Python-3.10.13 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7937MiB)
# Setup complete ✅ (16 CPUs, 31.1 GB RAM, 358.8/915.3 GB disk)

Ultralytics YOLOv8.2.22  Python-3.10.0 torch-2.1.2 CUDA:0 (Quadro RTX 4000, 8192MiB)
Setup complete  (20 CPUs, 63.7 GB RAM, 299.9/475.7 GB disk)


In [3]:
import pandas as pd
import numpy as np
cla=["complete","incomplete","blurry.complete","blurry.incomplete","hair"]
def process_dataframe(df):
    # Rename columns
    df.columns = ['class', 'x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4', 'confidence']

    # Calculate stomata center coordinates
    df['boundingbox_x'] = df[['x1', 'x2', 'x3', 'x4']].mean(axis=1)
    df['boundingbox_y'] = df[['y1', 'y2', 'y3', 'y4']].mean(axis=1)
    # Calculate width and height
    df['boundingbox_width'] = np.sqrt((df['x2'] - df['x1'])**2 + (df['y2'] - df['y1'])**2)
    df['boundingbox_height'] = np.sqrt((df['x4'] - df['x1'])**2 + (df['y4'] - df['y1'])**2)
    # Add type column
    df['File Name'] = i.split('\\')[-1].split('.')[0]
    df['class']=df['class'].apply(lambda x: cla[x])
        # Calculate rotation
    
    #df['rotation'] = np.degrees(np.arctan2(df['y2'] - df['y1'], df['x2'] - df['x1']))
    df = df.drop(columns=['x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4'])

    columns_order = ['class', 'boundingbox_x','boundingbox_y','boundingbox_width',
                     'boundingbox_height',
                     'confidence','File Name'
                     #,'rotation'
                     ]
    df = df[columns_order]
    return df



In [ ]:

import pandas as pd
import os
from tkinter.filedialog import askdirectory
from ultralytics import YOLO


# Put pretrain model weights in this file
model = YOLO('./K-FoldBest.pt')
path = askdirectory(title='Select Folder')
print(path.split('/')[-1])

results = model.predict( path,save=True,save_crop = False,save_txt = True,save_conf = True)  # predict on an image
def get_newest_folder(path):
  # Get a list of all directories in the specified path
  dirs = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
  # Sort the directories based on creation time
  dirs.sort(key=lambda d: os.path.getctime(os.path.join(path, d)), reverse=True)
  # Return the newest folder
  if dirs:
    return dirs[0]
  else:
    return None

# Example usage
path_to_search = "runs/obb/"
newest_folder = get_newest_folder(path_to_search)

labels = [os.path.join(path_to_search+newest_folder+'/labels', x) for x in os.listdir(path_to_search+newest_folder+'/labels') if x[-3:] == "txt"]
labels.sort()


#labels = [os.path.join('./runs/obb/predict/labels', x) for x in os.listdir('./runs/obb/predict/labels') if x[-3:] == "txt"]
#labels.sort()


# Convert and save the labels to csv file
dataframes = []
for i in labels:
    df = pd.read_csv(i, sep=" ",header = None)
    df= process_dataframe(df)
    save_name = i.replace('txt','csv')
    dataframes.append(df)

final_dataframe = pd.concat(dataframes, ignore_index=True)
#final_dataframe.to_excel('/content/drive/MyDrive/Ntu_stomata/result/res_noblurry.xlsx')
final_dataframe.to_csv('./result/Ntu/'+path.split('/')[-1]+'.csv', encoding = 'utf-8',index = False)
